# BBO Testing

In [1]:
# import bz2

# # Compressing a file
# with open('bbo-2024-41.csv', 'rb') as f:
#     content = f.read()
#     compressed_content = bz2.compress(content)

# with open('bbo-2024-41.csv.bz2', 'wb') as f:
#     f.write(compressed_content)


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('bbo_history.csv')
df

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus
0,2024-10-06 10:03:58.595406000,544,563,A,VXV4,16.00,50,0.0,0,Q
1,2024-10-06 10:03:58.595406000,627,635,A,VXV4,16.55,1,0.0,0,Q
2,2024-10-06 10:03:58.595406000,627,636,A,VXV4,16.55,2,0.0,0,Q
3,2024-10-06 10:03:58.595406000,627,637,A,VXV4,16.60,1,0.0,0,Q
4,2024-10-06 10:03:58.595406000,627,638,A,VXV4,16.60,2,0.0,0,Q
...,...,...,...,...,...,...,...,...,...,...
6429166,2024-10-11 16:00:01.575645000,1491615,1491615,D,VXH5,11.80,2,0.0,0,S
6429167,2024-10-11 16:00:01.575645000,1491618,1491618,D,VXMX4,0.00,0,0.0,0,S
6429168,2024-10-11 16:00:01.575645000,1491618,1491619,D,VXMV4,0.00,0,0.0,0,S
6429169,2024-10-11 16:00:01.575645000,1491618,1491620,D,VXF5,0.00,0,0.0,0,S


In [4]:
df.dtypes

Time              object
PktSeqNum          int64
MsgSeqNum          int64
MsgType           object
Symbol            object
BidPrice         float64
BidQuantity        int64
AskPrice         float64
AskQuantity        int64
TradingStatus     object
dtype: object

In [5]:
# df[(df['Symbol'].str.contains('ZVX'))&(df['TradingStatus'] == 'T')]

## Test 1: No Crossing when `TradingStatus` = `T`

In [5]:
df[(df['BidQuantity']* df['AskQuantity']>0)&(df['BidPrice']>df['AskPrice'])]

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus


In [6]:
df[(df['TradingStatus'] == 'T')&(df['BidQuantity']* df['AskQuantity']>0)&(df['BidPrice']>df['AskPrice'])]

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus


## Test 2: End-of-Day Best Bid/Offer with Zero Bid/Ask Quantity

In [7]:
last_rows = df.groupby('Symbol').tail(1)
last_rows

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus
186831,2024-10-07 06:04:12.052756000,431034,431034,D,ZVXX4,0.0,0,0.0,0,T
558618,2024-10-07 09:43:23.456924000,1103810,1103810,D,-VXV4+VXJ5,0.0,0,0.0,0,T
1776949,2024-10-07 14:55:34.959683000,3267125,3267125,D,-VXF5+VXJ5,0.0,0,0.0,0,T
1782720,2024-10-07 14:56:02.275473000,3278303,3278305,E,-VXTF5+VXTJ5,0.0,0,0.0,0,T
1816318,2024-10-07 14:58:42.000228000,3340761,3340765,D,VXTJ5,0.0,0,0.0,0,T
...,...,...,...,...,...,...,...,...,...,...
1898526,2024-10-07 16:00:01.346772000,3534257,3534257,D,VXF5,0.0,0,0.0,0,S
1898530,2024-10-07 16:00:01.346772000,3534261,3534261,D,VXG5,0.0,0,0.0,0,S
1898534,2024-10-07 16:00:01.346772000,3534265,3534265,D,VXH5,0.0,0,0.0,0,S
1898535,2024-10-07 16:00:01.346772000,3534265,3534266,D,VXMF5,0.0,0,0.0,0,S


In [8]:
last_rows[(last_rows['BidQuantity'] !=0) |(last_rows['AskQuantity'] !=0) | (last_rows['BidPrice'] !=0) | (last_rows['AskPrice'] !=0)]

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus


## Test 3: PktSeqNum, MsgSeqNum should be in order.

In [9]:
df['MsgSeqNum_diff'] = df['MsgSeqNum'] - df['MsgSeqNum'].shift(1)
df['PktSeqNum_diff'] = df['PktSeqNum'] - df['PktSeqNum'].shift(1)
df

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus,MsgSeqNum_diff,PktSeqNum_diff
0,2024-10-06 10:03:58.595406000,544,563,A,VXV4,16.00,50,0.00,0,Q,NaN,NaN
1,2024-10-06 10:03:58.595406000,627,635,A,VXV4,16.55,1,0.00,0,Q,72.0,83.0
2,2024-10-06 10:03:58.595406000,627,636,A,VXV4,16.55,2,0.00,0,Q,1.0,0.0
3,2024-10-06 10:03:58.595406000,627,637,A,VXV4,16.60,1,0.00,0,Q,1.0,0.0
4,2024-10-06 10:03:58.595406000,627,638,A,VXV4,16.60,2,0.00,0,Q,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1898532,2024-10-07 16:00:01.346772000,3534263,3534263,D,VXH5,0.00,0,20.80,1,S,1.0,1.0
1898533,2024-10-07 16:00:01.346772000,3534264,3534264,D,VXH5,0.00,0,20.85,1,S,1.0,1.0
1898534,2024-10-07 16:00:01.346772000,3534265,3534265,D,VXH5,0.00,0,0.00,0,S,1.0,1.0
1898535,2024-10-07 16:00:01.346772000,3534265,3534266,D,VXMF5,0.00,0,0.00,0,S,1.0,0.0


In [10]:
df[(df['MsgSeqNum_diff']<=0)|(df['PktSeqNum_diff']<0)]

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus,MsgSeqNum_diff,PktSeqNum_diff


## Test 4: At lease One of the four column BidPrice/BidQuantity/AskPrice/AskQuantity should be updated overtime for the same instrument

In [11]:
df_shift = df.copy()
df_shift[['BidPrice_diff', 'BidQuantity_diff', 'AskPrice_diff', 'AskQuantity_diff']] = df.groupby('Symbol')[['BidPrice', 'BidQuantity', 'AskPrice', 'AskQuantity']].diff().abs()
df_shift.dropna(inplace=True)
df_shift['diff'] = df_shift[['BidPrice_diff', 'BidQuantity_diff', 'AskPrice_diff', 'AskQuantity_diff']].sum(axis=1)
df_shift[df_shift['diff'] == 0]

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus,MsgSeqNum_diff,PktSeqNum_diff,BidPrice_diff,BidQuantity_diff,AskPrice_diff,AskQuantity_diff,diff


In [26]:
df[(df['Time']>='2024-10-07 14:45:49.445000000') & (df['Time']<='2024-10-07 14:45:49.500000')]

,Time,PktSeqNum,MsgSeqNum,MsgType,Symbol,BidPrice,BidQuantity,AskPrice,AskQuantity,TradingStatus,MsgSeqNum_diff,PktSeqNum_diff
1680461,2024-10-07 14:45:49.445136000,3087805,3087806,E,VXMV4,22.58,8,22.62,4,T,2.0,1.0
1680462,2024-10-07 14:45:49.445136000,3087805,3087807,E,VXMV4,22.58,6,22.62,4,T,1.0,0.0
1680463,2024-10-07 14:45:49.445136000,3087805,3087808,E,VXMV4,22.58,5,22.62,4,T,1.0,0.0
1680464,2024-10-07 14:45:49.445136000,3087805,3087809,E,VXMV4,22.57,61,22.62,4,T,1.0,0.0
1680465,2024-10-07 14:45:49.445136000,3087805,3087810,A,VXMV4,22.57,61,22.58,207,T,1.0,0.0
1680466,2024-10-07 14:45:49.445232000,3087812,3087812,E,VXMV4,22.57,61,22.62,4,T,2.0,7.0
1680467,2024-10-07 14:45:49.445254000,3087813,3087813,A,-VXV4+VXZ4,-2.70,4,-2.67,4,T,1.0,1.0
1680468,2024-10-07 14:45:49.445261000,3087813,3087814,A,VXMX4,20.71,3,20.76,4,T,1.0,0.0
1680469,2024-10-07 14:45:49.445267000,3087813,3087815,A,VXMV4,22.57,61,22.60,2,T,1.0,0.0
1680470,2024-10-07 14:45:49.445270000,3087813,3087816,D,VXMV4,22.57,60,22.60,2,T,1.0,0.0


In [ ]:
import pandas as pd
df_correct = pd.read_csv(f'../UpdateOrderBook/bbo-2024-41.csv')
df = pd.read_csv('bbo_history.csv')

# Compare the two CSV files
differences = df_correct.compare(df)

if differences.empty:
    print("The two CSV files are equal.")
else:
    # Display the first 10 differences
    print("Differences between the two CSV files:")
    differences.head(10)


In [ ]:
import pandas as pd
df_arb = pd.read_csv('arb_bbo_history.csv')

# Compare the two CSV files
differences = df_correct.compare(df)

if differences.empty:
    print("The two CSV files are equal.")
else:
    # Display the first 10 differences
    print("Differences between the two CSV files:")
    differences.head(10)


The two CSV files are equal.
